# CNN Classification with MPQA Dataset
<hr>

The __modus operandi__ for text classification is to use __word embedding__ for representing words and a Convolutional neural network to learn how to discriminate documents on classification problems. 

__Yoav Goldberg__ commented in _A Primer on Neural Network Models for Natural Language Processing, 2015._ :
> _The non-linearity of the network, as well as the ability to easily integrate pre-trained
word embeddings, often lead to superior classification accuracy._

He also commented in _Neural Network Methods for Natural Language Processing, 2017_ :
> ... _the CNN is in essence a feature-extracting architecture. ... . The CNNs layer's responsibility is to extract meaningful sub-structures that are useful for the overall prediction task at hand._

We will build a text classification model using CNN model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 1,2, 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2) constraint: 3.
- Batch Size: 50.
- Update Rule: Adam

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [2]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


,sentence,label,split
0,"smart and alert , thirteen conversations about...",0,train
1,"color , musical bounce and warm seas lapping o...",0,train
2,it is not a mass market entertainment but an u...,0,train
3,a light hearted french film about the spiritua...,0,train
4,my wife is an actress has its moments in looki...,0,train
...,...,...,...
9995,"in the end , they discover that balance in lif...",1,train
9996,a counterfeit 1000 tomin bank note is passed i...,1,train
9997,enter the beautiful and mysterious secret agen...,1,train
9998,after listening to a missionary from china spe...,1,train


In [3]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [4]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5000,5000
1,5000,5000


In [5]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [6]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [7]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

print("Example of sentence: ", sentences[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          6397200   
_________________________________________________________________
conv1d (Conv1D)              (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 4900)              0         
_________________________________________________________________
dropout (Dropout)            (None, 4900)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                49010     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0

In [13]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [14]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=15, verbose=2, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

Epoch 1/15
180/180 - 19s - loss: 0.6851 - accuracy: 0.5187 - val_loss: 0.5145 - val_accuracy: 0.8610
Epoch 2/15
180/180 - 18s - loss: 0.3089 - accuracy: 0.8681 - val_loss: 0.2200 - val_accuracy: 0.9140
Epoch 3/15
180/180 - 17s - loss: 0.1218 - accuracy: 0.9559 - val_loss: 0.2579 - val_accuracy: 0.9090
Epoch 4/15
180/180 - 17s - loss: 0.0689 - accuracy: 0.9776 - val_loss: 0.3511 - val_accuracy: 0.9110
Epoch 5/15
180/180 - 18s - loss: 0.0463 - accuracy: 0.9864 - val_loss: 0.4157 - val_accuracy: 0.9040
Epoch 6/15
180/180 - 17s - loss: 0.0361 - accuracy: 0.9904 - val_loss: 0.4922 - val_accuracy: 0.9000
Epoch 7/15
180/180 - 17s - loss: 0.0322 - accuracy: 0.9892 - val_loss: 0.6074 - val_accuracy: 0.9090
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 91.39999747276306
Epoch 1/15
180/180 - 19s - loss: 0.5219 - accuracy: 0.7141 - val_loss: 0.2361 - val_accuracy: 0.9030
Epoch 2/15
180/180 - 18s - loss: 0.2007 - accuracy: 0.9270 - val_loss: 0.20

Epoch 5/15
180/180 - 18s - loss: 0.0494 - accuracy: 0.9768 - val_loss: 0.5339 - val_accuracy: 0.8960
Epoch 6/15
180/180 - 18s - loss: 0.0427 - accuracy: 0.9814 - val_loss: 0.5424 - val_accuracy: 0.8960
Epoch 7/15
180/180 - 18s - loss: 0.0345 - accuracy: 0.9829 - val_loss: 0.6695 - val_accuracy: 0.9030
Epoch 8/15
180/180 - 18s - loss: 0.0350 - accuracy: 0.9840 - val_loss: 0.6754 - val_accuracy: 0.8950
Epoch 9/15
180/180 - 18s - loss: 0.0327 - accuracy: 0.9836 - val_loss: 0.6963 - val_accuracy: 0.8960
Epoch 10/15
180/180 - 18s - loss: 0.0343 - accuracy: 0.9828 - val_loss: 0.7160 - val_accuracy: 0.8970
Epoch 11/15
180/180 - 18s - loss: 0.0319 - accuracy: 0.9847 - val_loss: 0.7615 - val_accuracy: 0.8920
Epoch 12/15
180/180 - 18s - loss: 0.0308 - accuracy: 0.9844 - val_loss: 0.8657 - val_accuracy: 0.8970
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 90.2999997138977

  Activation Filters       acc1  acc2  acc3  acc4       acc5       acc6 

Test Accuracy: 92.69999861717224
Epoch 1/15
180/180 - 20s - loss: 0.5901 - accuracy: 0.6478 - val_loss: 0.3184 - val_accuracy: 0.8930
Epoch 2/15
180/180 - 19s - loss: 0.3255 - accuracy: 0.9028 - val_loss: 0.2560 - val_accuracy: 0.9210
Epoch 3/15
180/180 - 19s - loss: 0.1935 - accuracy: 0.9436 - val_loss: 0.2204 - val_accuracy: 0.9230
Epoch 4/15
180/180 - 19s - loss: 0.1227 - accuracy: 0.9606 - val_loss: 0.2648 - val_accuracy: 0.9150
Epoch 5/15
180/180 - 18s - loss: 0.1036 - accuracy: 0.9644 - val_loss: 0.3502 - val_accuracy: 0.9130
Epoch 6/15
180/180 - 19s - loss: 0.0958 - accuracy: 0.9670 - val_loss: 0.3776 - val_accuracy: 0.9040
Epoch 7/15
180/180 - 18s - loss: 0.0964 - accuracy: 0.9633 - val_loss: 0.3954 - val_accuracy: 0.9140
Epoch 8/15
180/180 - 18s - loss: 0.0934 - accuracy: 0.9644 - val_loss: 0.3929 - val_accuracy: 0.9160
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 92.29999780654907
Epoch 1/15
180/180 - 20s - loss: 0.5481 - 

Epoch 5/15
180/180 - 20s - loss: 0.0334 - accuracy: 0.9816 - val_loss: 0.4313 - val_accuracy: 0.9130
Epoch 6/15
180/180 - 20s - loss: 0.0295 - accuracy: 0.9832 - val_loss: 0.4829 - val_accuracy: 0.9130
Epoch 7/15
180/180 - 20s - loss: 0.0285 - accuracy: 0.9812 - val_loss: 0.5717 - val_accuracy: 0.9060
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 92.59999990463257
Epoch 1/15
180/180 - 21s - loss: 0.5733 - accuracy: 0.6467 - val_loss: 0.2467 - val_accuracy: 0.9000
Epoch 2/15
180/180 - 20s - loss: 0.2482 - accuracy: 0.9192 - val_loss: 0.1988 - val_accuracy: 0.9230
Epoch 3/15
180/180 - 20s - loss: 0.1184 - accuracy: 0.9672 - val_loss: 0.2618 - val_accuracy: 0.9100
Epoch 4/15
180/180 - 20s - loss: 0.0715 - accuracy: 0.9804 - val_loss: 0.3669 - val_accuracy: 0.9120
Epoch 5/15
180/180 - 20s - loss: 0.0603 - accuracy: 0.9827 - val_loss: 0.4044 - val_accuracy: 0.9090
Epoch 6/15
180/180 - 20s - loss: 0.0529 - accuracy: 0.9842 - val_loss: 0.44

Epoch 2/15
180/180 - 19s - loss: 0.2021 - accuracy: 0.9343 - val_loss: 0.2348 - val_accuracy: 0.9040
Epoch 3/15
180/180 - 19s - loss: 0.0988 - accuracy: 0.9680 - val_loss: 0.3252 - val_accuracy: 0.9030
Epoch 4/15
180/180 - 19s - loss: 0.0709 - accuracy: 0.9799 - val_loss: 0.4325 - val_accuracy: 0.9020
Epoch 5/15
180/180 - 19s - loss: 0.0584 - accuracy: 0.9822 - val_loss: 0.5256 - val_accuracy: 0.8930
Epoch 6/15
180/180 - 19s - loss: 0.0511 - accuracy: 0.9820 - val_loss: 0.5402 - val_accuracy: 0.8960
Epoch 7/15
180/180 - 19s - loss: 0.0502 - accuracy: 0.9842 - val_loss: 0.4944 - val_accuracy: 0.9050
Epoch 8/15
180/180 - 19s - loss: 0.0522 - accuracy: 0.9827 - val_loss: 0.6147 - val_accuracy: 0.8920
Epoch 9/15
180/180 - 19s - loss: 0.0471 - accuracy: 0.9842 - val_loss: 0.7574 - val_accuracy: 0.8840
Epoch 10/15
180/180 - 19s - loss: 0.0503 - accuracy: 0.9818 - val_loss: 0.6477 - val_accuracy: 0.8990
Epoch 11/15
180/180 - 19s - loss: 0.0472 - accuracy: 0.9828 - val_loss: 0.6203 - val_accur

...

Test Accuracy: 92.1999990940094
Epoch 1/15
180/180 - 21s - loss: 0.4624 - accuracy: 0.7603 - val_loss: 0.2487 - val_accuracy: 0.9060
Epoch 2/15
180/180 - 20s - loss: 0.1517 - accuracy: 0.9576 - val_loss: 0.2494 - val_accuracy: 0.9070
Epoch 3/15
180/180 - 20s - loss: 0.0612 - accuracy: 0.9893 - val_loss: 0.2864 - val_accuracy: 0.9080
Epoch 4/15
180/180 - 20s - loss: 0.0322 - accuracy: 0.9970 - val_loss: 0.3203 - val_accuracy: 0.9110
Epoch 5/15
180/180 - 20s - loss: 0.0210 - accuracy: 0.9980 - val_loss: 0.3917 - val_accuracy: 0.8970
Epoch 6/15
180/180 - 20s - loss: 0.0152 - accuracy: 0.9994 - val_loss: 0.3946 - val_accuracy: 0.9060
Epoch 7/15
180/180 - 20s - loss: 0.0114 - accuracy: 0.9993 - val_loss: 0.4193 - val_accuracy: 0.9040
Epoch 8/15
180/180 - 20s - loss: 0.0099 - accuracy: 0.9991 - val_loss: 0.4608 - val_accuracy: 0.9060
Epoch 9/15
180/180 - 20s - loss: 0.0078 - accuracy: 0.9996 - val_loss: 0.4894 - val_accuracy: 0.9040
Restoring model weights from the end of the best epoch.
Epo

180/180 - 20s - loss: 0.0093 - accuracy: 0.9996 - val_loss: 0.4734 - val_accuracy: 0.8900
Epoch 7/15
180/180 - 21s - loss: 0.0074 - accuracy: 0.9996 - val_loss: 0.5084 - val_accuracy: 0.8900
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 90.20000100135803
Epoch 1/15
180/180 - 21s - loss: 0.4336 - accuracy: 0.7779 - val_loss: 0.2322 - val_accuracy: 0.9110
Epoch 2/15
180/180 - 21s - loss: 0.1306 - accuracy: 0.9628 - val_loss: 0.2167 - val_accuracy: 0.9130
Epoch 3/15
180/180 - 21s - loss: 0.0414 - accuracy: 0.9936 - val_loss: 0.2599 - val_accuracy: 0.9210
Epoch 4/15
180/180 - 20s - loss: 0.0198 - accuracy: 0.9978 - val_loss: 0.3441 - val_accuracy: 0.9050
Epoch 5/15
180/180 - 20s - loss: 0.0142 - accuracy: 0.9988 - val_loss: 0.3309 - val_accuracy: 0.9150
Epoch 6/15
180/180 - 20s - loss: 0.0105 - accuracy: 0.9991 - val_loss: 0.4047 - val_accuracy: 0.9080
Epoch 7/15
180/180 - 20s - loss: 0.0098 - accuracy: 0.9987 - val_loss: 0.4137 - val_ac

Epoch 7/15
180/180 - 22s - loss: 0.0061 - accuracy: 0.9996 - val_loss: 0.3492 - val_accuracy: 0.9180
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 93.30000281333923
Epoch 1/15
180/180 - 23s - loss: 0.4425 - accuracy: 0.7703 - val_loss: 0.2433 - val_accuracy: 0.8990
Epoch 2/15
180/180 - 22s - loss: 0.1391 - accuracy: 0.9584 - val_loss: 0.2033 - val_accuracy: 0.9230
Epoch 3/15
180/180 - 22s - loss: 0.0450 - accuracy: 0.9929 - val_loss: 0.2544 - val_accuracy: 0.9090
Epoch 4/15
180/180 - 22s - loss: 0.0196 - accuracy: 0.9983 - val_loss: 0.2836 - val_accuracy: 0.9170
Epoch 5/15
180/180 - 21s - loss: 0.0134 - accuracy: 0.9993 - val_loss: 0.3285 - val_accuracy: 0.9150
Epoch 6/15
180/180 - 22s - loss: 0.0109 - accuracy: 0.9992 - val_loss: 0.3459 - val_accuracy: 0.9170
Epoch 7/15
180/180 - 21s - loss: 0.0081 - accuracy: 0.9996 - val_loss: 0.3692 - val_accuracy: 0.9180
Restoring model weights from the end of the best epoch.
Epoch 00007: early 

Epoch 1/15
180/180 - 24s - loss: 0.4424 - accuracy: 0.7682 - val_loss: 0.2176 - val_accuracy: 0.9130
Epoch 2/15
180/180 - 23s - loss: 0.1263 - accuracy: 0.9649 - val_loss: 0.2321 - val_accuracy: 0.9190
Epoch 3/15
180/180 - 23s - loss: 0.0434 - accuracy: 0.9933 - val_loss: 0.2647 - val_accuracy: 0.9120
Epoch 4/15
180/180 - 22s - loss: 0.0176 - accuracy: 0.9990 - val_loss: 0.3189 - val_accuracy: 0.9150
Epoch 5/15
180/180 - 23s - loss: 0.0131 - accuracy: 0.9990 - val_loss: 0.3680 - val_accuracy: 0.9080
Epoch 6/15
180/180 - 23s - loss: 0.0092 - accuracy: 0.9994 - val_loss: 0.4061 - val_accuracy: 0.9110
Epoch 7/15
180/180 - 22s - loss: 0.0070 - accuracy: 0.9997 - val_loss: 0.4272 - val_accuracy: 0.9150
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 91.90000295639038
Epoch 1/15
180/180 - 24s - loss: 0.4358 - accuracy: 0.7762 - val_loss: 0.2455 - val_accuracy: 0.9040
Epoch 2/15
180/180 - 23s - loss: 0.1303 - accuracy: 0.9629 - val_loss: 0.25

Epoch 6/15
180/180 - 23s - loss: 0.0144 - accuracy: 0.9994 - val_loss: 0.3768 - val_accuracy: 0.9130
Epoch 7/15
180/180 - 23s - loss: 0.0107 - accuracy: 0.9993 - val_loss: 0.4014 - val_accuracy: 0.9100
Epoch 8/15
180/180 - 23s - loss: 0.0098 - accuracy: 0.9996 - val_loss: 0.4394 - val_accuracy: 0.9070
Epoch 9/15
180/180 - 23s - loss: 0.0124 - accuracy: 0.9970 - val_loss: 0.4309 - val_accuracy: 0.9050
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 92.00000166893005
Epoch 1/15
180/180 - 24s - loss: 0.4353 - accuracy: 0.7749 - val_loss: 0.2560 - val_accuracy: 0.8990
Epoch 2/15
180/180 - 23s - loss: 0.1411 - accuracy: 0.9602 - val_loss: 0.2175 - val_accuracy: 0.9210
Epoch 3/15
180/180 - 23s - loss: 0.0493 - accuracy: 0.9910 - val_loss: 0.2561 - val_accuracy: 0.9220
Epoch 4/15
180/180 - 23s - loss: 0.0268 - accuracy: 0.9972 - val_loss: 0.2660 - val_accuracy: 0.9250
Epoch 5/15
180/180 - 23s - loss: 0.0171 - accuracy: 0.9993 - val_loss: 0.30

## Summary

In [15]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
2,relu,3,91.600001,92.100000,91.500002,91.900003,90.499997,92.600000,92.299998,92.400002,92.500001,93.099999,92.050000
6,tanh,1,92.299998,92.400002,89.700001,91.900003,92.699999,91.299999,90.399998,92.600000,92.500001,93.800002,91.960000
0,relu,1,91.399997,92.600000,92.100000,91.700000,92.199999,92.299998,92.900002,91.700000,91.600001,90.300000,91.880000
1,relu,2,90.899998,92.000002,91.500002,92.400002,91.600001,91.600001,91.299999,92.699999,92.299998,92.000002,91.830000
10,tanh,5,91.799998,93.300003,92.299998,91.399997,90.899998,92.500001,91.799998,91.100001,90.799999,92.199999,91.809999
8,tanh,3,91.500002,92.000002,91.299999,93.000001,90.300000,91.900003,92.699999,91.600001,92.199999,91.100001,91.760001
5,relu,6,92.799997,91.600001,90.899998,91.900003,91.700000,93.099999,91.900003,91.100001,90.300000,91.600001,91.690000
7,tanh,2,91.700000,91.100001,90.899998,91.900003,91.299999,93.099999,92.299998,91.299999,91.200000,91.900003,91.670000
4,relu,5,91.100001,91.900003,90.799999,91.700000,92.699999,92.500001,91.700000,90.899998,91.200000,91.799998,91.630000
9,tanh,4,91.900003,90.700001,90.600002,92.500001,91.700000,90.200001,92.100000,90.799999,93.199998,92.100000,91.580001


In [16]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,92.05
tanh,91.96


In [17]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_SUBJ.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [18]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [19]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [20]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [21]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [22]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [23]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.93121508,  0.34473418, -0.70878697, ...,  0.25505658,
         0.08051993,  0.51291611],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## CNN Model

In [24]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [25]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(vocab_size, 300))
model_0.summary()

Model: "sequential_121"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_121 (Embedding)    (None, 100, 300)          6089700   
_________________________________________________________________
conv1d_121 (Conv1D)          (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_121 (MaxPoolin (None, 49, 100)           0         
_________________________________________________________________
flatten_121 (Flatten)        (None, 4900)              0         
_________________________________________________________________
dropout_242 (Dropout)        (None, 4900)              0         
_________________________________________________________________
dense_242 (Dense)            (None, 10)                49010     
_________________________________________________________________
dropout_243 (Dropout)        (None, 10)             

## Train and Test the Model

In [26]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [27]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=30, verbose=0, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 89.99999761581421
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 88.30000162124634
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 84.79999899864197
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 89.49999809265137
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 85.50000190734863
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 80.19999861717224
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 85.19999980926514
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 90.10000228881836
Restoring model weights from the end of the best epoch.
Epoch 00

Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 85.10000109672546
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 88.89999985694885
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 86.10000014305115
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 88.7000024318695
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 84.79999899864197
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 76.49999856948853
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 86.2999975681305
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 88.7000024318695
Restoring model weights from the end of the best epoch.
Epoch 00007

## Summary

In [28]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
2,relu,3,86.799997,88.099998,89.200002,89.099997,85.699999,88.099998,91.000003,87.599999,88.900000,82.999998,87.749999
0,relu,1,89.999998,88.300002,84.799999,89.499998,85.500002,80.199999,85.200000,90.100002,89.099997,91.399997,87.409999
3,relu,4,87.199998,85.000002,85.799998,87.099999,90.799999,88.999999,85.600001,87.000000,79.500002,85.600001,86.260000
1,relu,2,88.599998,87.400001,84.799999,83.899999,89.800000,81.199998,90.700001,83.399999,88.700002,80.500001,85.900000
6,relu,7,80.900002,87.800002,88.499999,77.899998,82.999998,84.799999,84.700000,89.499998,85.299999,86.100000,84.850000
4,relu,5,79.699999,84.500003,88.999999,88.499999,86.299998,84.799999,83.399999,83.200002,88.200003,79.900002,84.750000
7,relu,8,86.699998,84.500003,84.399998,82.099998,88.000000,87.599999,85.600001,71.899998,85.399997,88.099998,84.429999
5,relu,6,85.100001,88.900000,86.100000,88.700002,84.799999,76.499999,86.299998,88.700002,76.300001,82.099998,84.350000


In [29]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,87.749999


In [30]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_SUBJ_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [31]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                 input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Conv1D(filters=filters, kernel_size = kernel_size, activation = activation, 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=activation, 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [32]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(vocab_size, 300))
model_0.summary()

Model: "sequential_202"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_202 (Embedding)    (None, 100, 300)          6088200   
_________________________________________________________________
conv1d_202 (Conv1D)          (None, 98, 100)           90100     
_________________________________________________________________
max_pooling1d_202 (MaxPoolin (None, 49, 100)           0         
_________________________________________________________________
flatten_202 (Flatten)        (None, 4900)              0         
_________________________________________________________________
dropout_404 (Dropout)        (None, 4900)              0         
_________________________________________________________________
dense_404 (Dense)            (None, 10)                49010     
_________________________________________________________________
dropout_405 (Dropout)        (None, 10)             

## Train and Test the Model

In [33]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [34]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=20, verbose=0, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 91.69999957084656
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 91.79999828338623
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 89.49999809265137
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 88.59999775886536
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 92.29999780654907
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 90.39999842643738
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 90.10000228881836
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 91.39999747276306
Restoring model weights from the end of the best epoch.
Epoch 00

Test Accuracy: 89.49999809265137
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 91.20000004768372
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 88.30000162124634
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 91.29999876022339
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 89.80000019073486
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 90.79999923706055
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 93.30000281333923
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 88.7000024318695
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 89.3999993801117
Restoring model weights from the 

## Summary

In [35]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,91.700000,91.799998,89.499998,88.599998,92.299998,90.399998,90.100002,91.399997,91.500002,91.600001,90.889999
1,relu,2,88.800001,92.400002,90.799999,91.900003,91.299999,91.900003,92.400002,86.100000,91.600001,91.500002,90.870001
6,relu,7,92.699999,91.399997,91.600001,91.100001,90.600002,92.299998,87.300003,90.100002,90.200001,91.100001,90.840001
4,relu,5,90.399998,92.199999,88.900000,91.000003,88.599998,90.300000,93.199998,89.700001,91.700000,92.100000,90.810000
2,relu,3,92.000002,91.500002,91.600001,89.800000,91.399997,93.900001,89.099997,91.799998,88.800001,86.199999,90.610000
7,relu,8,88.999999,90.799999,88.999999,90.799999,90.899998,91.399997,89.099997,90.899998,92.400002,91.600001,90.589999
5,relu,6,89.499998,91.200000,88.300002,91.299999,89.800000,90.799999,93.300003,88.700002,89.399999,90.100002,90.240000
3,relu,4,89.800000,91.799998,88.700002,91.299999,91.299999,86.299998,91.799998,90.799999,86.900002,91.600001,90.030000


In [36]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('CNN_SUBJ_3.xlsx', sheet_name='dynamic')